In [3]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from PIL import Image, ImageTk, ImageDraw, ImageFont
import numpy as np
from datetime import datetime
import smtplib


# Monate auf Deutsch
MONATE = [
    "Januar", "Februar", "März", "April", "Mai", "Juni",
    "Juli", "August", "September", "Oktober", "November", "Dezember"
]

# Funktion zur Berechnung des Sternzeichens
def get_zodiac_sign(day, month):
    """
    Berechnet das Sternzeichen basierend auf Tag und Monat.
    Args:
        day: Tag des Geburtsdatums (int).
        month: Monat des Geburtsdatums (int).
    Returns:
        str: Name des Sternzeichens.
    """
    month_names = {
        1: "Januar",
        2: "Februar",
        3: "März",
        4: "April",
        5: "Mai",
        6: "Juni",
        7: "Juli",
        8: "August",
        9: "September",
        10: "Oktober",
        11: "November",
        12: "Dezember"
    }

    if (month == 12 and day >= 22) or (month == 1 and day <= 19):
        return "Steinbock"
    elif (month == 1 and day >= 20) or (month == 2 and day <= 18):
        return "Wassermann"
    elif (month == 2 and day >= 19) or (month == 3 and day <= 20):
        return "Fisch"
    elif (month == 3 and day >= 21) or (month == 4 and day <= 19):
        return "Widder"
    elif (month == 4 and day >= 20) or (month == 5 and day <= 20):
        return "Stier"
    elif (month == 5 and day >= 21) or (month == 6 and day <= 20):
        return "Zwilling"
    elif (month == 6 and day >= 21) or (month == 7 and day <= 22):
        return "Krebs"
    elif (month == 7 and day >= 23) or (month == 8 and day <= 22):
        return "Löwe"
    elif (month == 8 and day >= 23) or (month == 9 and day <= 22):
        return "Jungfrau"
    elif (month == 9 and day >= 23) or (month == 10 and day <= 22):
        return "Waage"
    elif (month == 10 and day >= 23) or (month == 11 and day <= 21):
        return "Skorpion"
    elif (month == 11 and day >= 22) or (month == 12 and day <= 21):
        return "Schütze"
    else:
        # Ungültiges Datum
        print(f"Ungültiges Geburtsdatum: {day}.{month_names[month]}")
        return None  # Oder alternativ ein Fehler-Wert zurückgeben

# Bildauswahl-Funktion
def select_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        try:
            image = Image.open(file_path)
            image.thumbnail((800, 800), Image.LANCZOS)
            user_image = ImageTk.PhotoImage(image)
            image_label.config(image=user_image)
            image_label.image = user_image
            image_label.file_path = file_path
        except Exception as e:
            messagebox.showerror("Fehler", f"Bild konnte nicht geladen werden: {e}")

# Bildgröße anpassen und zuschneiden
def resize_and_crop_to_800x800(image_array):
    image = Image.fromarray(image_array)
    width, height = image.size
    aspect_ratio = width / height

    if (aspect_ratio > 1):  # Landscape orientation
        new_height = 800
        new_width = int(aspect_ratio * new_height)
    else:  # Portrait orientation or square
        new_width = 800
        new_height = int(new_width / aspect_ratio)

    image = image.resize((new_width, new_height), Image.LANCZOS)
    left = (new_width - 800) / 2
    top = (new_height - 800) / 2
    right = (new_width + 800) / 2
    bottom = (new_height + 800) / 2

    image = image.crop((left, top, right, bottom))
    return np.array(image)


# Ergebnisse anzeigen und E-Mail versenden
def show_results():
    name = name_entry.get()
    day = int(day_var.get())
    month = month_menu.current() + 1  # Get the actual month number (January = 1)
    month_name = MONATE[month - 1]  # Get the corresponding month name
    year = int(year_var.get())
    image_path = getattr(image_label, 'file_path', None)

    try:
        birth_date_obj = datetime(year, month, day)
        zodiac_sign = get_zodiac_sign(day, month)
    except ValueError:
        messagebox.showerror("Fehler", "Ungültiges Geburtsdatum. Bitte korrektes Datum eingeben.")
        return

    if not name or not email or not day or not month or not year or not image_path:
        messagebox.showerror("Fehler", "Bitte füllen Sie alle Felder aus und wählen Sie ein Bild aus.")
        return

    try:
        user_image = Image.open(image_path)
        user_image = user_image.convert("RGBA")
        user_image_array = np.array(user_image)
        user_image_array = resize_and_crop_to_800x800(user_image_array)
        user_image = Image.fromarray(user_image_array)
        user_image_width, user_image_height = user_image.size
    except Exception as e:
        messagebox.showerror("Fehler", f"Bild konnte nicht verarbeitet werden: {e}")
        return

    zodiac_image_path = f"zodiac_images/{zodiac_sign.lower()}.png"
    try:
        zodiac_image = Image.open(zodiac_image_path)
        zodiac_image.thumbnail((200, 200), Image.LANCZOS)
        zodiac_image = zodiac_image.convert("RGBA")

        # Alphakanal anpassen, um Transparenz hinzuzufügen
        alpha = zodiac_image.split()[3]
        alpha = Image.eval(alpha, lambda a: int(a * 0.5))
        zodiac_image.putalpha(alpha)

        zodiac_image_width, zodiac_image_height = zodiac_image.size
    except FileNotFoundError:
        messagebox.showerror("Fehler", f"Sternzeichenbild für {zodiac_sign} nicht gefunden.")
        return

    combined_image = Image.new('RGBA', (user_image_width, user_image_height))
    combined_image.paste(user_image, (0, 0))
    combined_image.paste(zodiac_image, (user_image_width - zodiac_image_width, 0), zodiac_image)

    draw = ImageDraw.Draw(combined_image)
    try:
        font = ImageFont.truetype("arial.ttf", 40)
    except IOError:
        font = ImageFont.load_default()

    text = f"Name: {name}\nGeburtsdatum: {day:02d}.{month_name}.{year}\nDu bist: {zodiac_sign}"
    text_position = (10, user_image_height - 130)
    draw.text(text_position, text, font=font, fill="white")

    save_path = filedialog.asksaveasfilename(defaultextension=".png",
                                             filetypes=[("PNG files", "*.png"), ("All files", "*.*")])

    if save_path:
        try:
            combined_image.save(save_path)
            messagebox.showinfo("Erfolg", f"Bild erfolgreich gespeichert unter: {save_path}")
        
            
        except Exception as e:
            messagebox.showerror("Fehler", f"Bild konnte nicht gespeichert werden: {e}")

    combined_image.show()

# GUI-Setup
root = tk.Tk()
root.title("Sternzeichen Bestimmung")

# Layout
name_label = ttk.Label(root, text="Name:")
name_label.grid(row=0, column=0, padx=10, pady=10)
name_entry = ttk.Entry(root)
name_entry.grid(row=0, column=1, padx=10, pady=10, columnspan=3)


birth_date_label = ttk.Label(root, text="Geburtsdatum:")
birth_date_label.grid(row=2, column=0, padx=10, pady=10)

# Tag-Pulldown-Menü
day_label = ttk.Label(root, text="Tag:")
day_label.grid(row=2, column=1, padx=10, pady=10)
day_var = tk.StringVar()
day_menu = ttk.Combobox(root, textvariable=day_var, values=[f"{i:02d}" for i in range(1, 32)])
day_menu.grid(row=2, column=2, padx=10, pady=10)
day_menu.current(0)

# Monat-Pulldown-Menü
month_label = ttk.Label(root, text="Monat:")
month_label.grid(row=2, column=3, padx=10, pady=10)
month_var = tk.StringVar()
month_menu = ttk.Combobox(root, textvariable=month_var, values=MONATE)
month_menu.grid(row=2, column=4, padx=10, pady=10)
month_menu.current(0)  # Initial month selection set to January (index 0)

# Jahr-Pulldown-Menü
year_label = ttk.Label(root, text="Jahr:")
year_label.grid(row=2, column=5, padx=10, pady=10)
year_var = tk.StringVar()
year_menu = ttk.Combobox(root, textvariable=year_var, values=[str(i) for i in range(1900, datetime.now().year + 1)])
year_menu.grid(row=2, column=6, padx=10, pady=10)
year_menu.current(len(year_menu['values']) - 1)

# Bildanzeige
image_label = ttk.Label(root)
image_label.grid(row=3, column=0, columnspan=7, padx=10, pady=10)

# Bildauswahl-Button
select_image_button = ttk.Button(root, text="Bild auswählen", command=select_image)
select_image_button.grid(row=4, column=0, columnspan=7, padx=10, pady=10)

# Ergebnis-Button
show_results_button = ttk.Button(root, text="Ergebnisse anzeigen", command=show_results)
show_results_button.grid(row=5, column=0, columnspan=7, padx=10, pady=10)

# Hauptschleife starten
root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Admin\miniconda3\envs\DataCraft2\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_1152\2830929804.py", line 122, in show_results
    if not name or not email or not day or not month or not year or not image_path:
                       ^^^^^
NameError: name 'email' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Admin\miniconda3\envs\DataCraft2\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_1152\2830929804.py", line 122, in show_results
    if not name or not email or not day or not month or not year or not image_path:
                       ^^^^^
NameError: name 'email' is not defined
